In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv(r'E:\Corelia\Final Project\art_coverage_by_country_clean.csv')
df2 = pd.read_csv(r'E:\Corelia\Final Project\art_pediatric_coverage_by_country_clean.csv')
df3 = pd.read_csv(r'E:\Corelia\Final Project\no_of_cases_adults_15_to_49_by_country_clean.csv')
df4 = pd.read_csv(r'E:\Corelia\Final Project\no_of_deaths_by_country_clean.csv')
df5 = pd.read_csv(r'E:\Corelia\Final Project\no_of_people_living_with_hiv_by_country_clean.csv')
df6 = pd.read_csv(r'E:\Corelia\Final Project\prevention_of_mother_to_child_transmission_by_country_clean.csv')

In [3]:
import re

# 1) تنظيف عمود "Reported number of people receiving ART"
df1["Reported number of people receiving ART"] = (
    df1["Reported number of people receiving ART"]
        .astype(str)
        .replace({"—": None, "Nodata": None, "No data": None, "no data": None})
        .str.replace(",", "", regex=False)
        .replace("nan", None)
        .astype(float)
)

# 2) دالة لفصل الأعمدة اللي فيها range (قيم بالشكل "100000 [90000–110000]")
def split_range(series):
    cleaned = (series.astype(str)
                       .str.replace("–", "-", regex=False)
                       .str.replace(",", "", regex=False))
    pattern = r'(?P<median>\d+\.?\d*)\s*\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    out = cleaned.str.extract(pattern).astype(float)
    return out

# 3) الأعمدة اللي محتاجة تفكيك
cols_range = [
    "Estimated number of people living with HIV",
    "Estimated ART coverage among people living with HIV (%)"
]

for col in cols_range:
    parts = split_range(df1[col])
    for p in ['median', 'min', 'max']:
        df1[f"{col}_{p}"] = parts[p]
    df1.drop(columns=col, inplace=True)  # حذف العمود الأصلي بعد الفصل

# 4) مؤشرات إضافية
df1["Coverage_calc_%"] = (
    100 * df1["Reported number of people receiving ART"] /
    df1["Estimated number of people living with HIV_median"]
)

df1["Treatment_gap"] = (
    df1["Estimated number of people living with HIV_median"] -
    df1["Reported number of people receiving ART"]
)

# 5) حفظ النسخة النهائية
import os

clean_dir = r"E:\Corelia\Final Project\HIV\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "art_coverage_clean_numeric.csv")
df1.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned file saved to: {output_path}")


✔ Cleaned file saved to: E:\Corelia\Final Project\HIV\clean\art_coverage_clean_numeric.csv


In [6]:
art_pedia_path = r"E:\Corelia\Final Project\art_pediatric_coverage_by_country_clean.csv"
df2 = pd.read_csv(art_pedia_path, encoding='utf-8')


In [7]:
import pandas as pd
import numpy as np
import os
import re

# ✅ 1) تحميل الملف
art_pedia_path = r"E:\Corelia\Final Project\art_pediatric_coverage_by_country_clean.csv"
df2 = pd.read_csv(art_pedia_path, encoding='utf-8')

# ✅ 2) تنظيف عمود "Reported number of children receiving ART"
df2["Reported number of children receiving ART"] = (
    df2["Reported number of children receiving ART"]
        .astype(str)
        .replace({"—": None, "Nodata": None, "No data": None, "no data": None})
        .str.replace(" ", "", regex=False)              # ← إزالة المسافات
        .str.replace(",", "", regex=False)              # ← إزالة الفواصل
        .replace("nan", None)
        .astype(float)
)

# ✅ 3) دالة لفصل الأعمدة اللي فيها Range بالشكل "1800 [1600–2100]"
def split_range(series):
    cleaned = (series.astype(str)
                       .str.replace("–", "-", regex=False)
                       .str.replace(" ", "", regex=False)
                       .str.replace(",", "", regex=False))
    pattern = r'(?P<median>\d+\.?\d*)\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    return cleaned.str.extract(pattern).astype(float)

# ✅ 4) الأعمدة اللي فيها range
cols_range = [
    "Estimated number of children needing ART based on WHO methods",
    "Estimated ART coverage among children (%)"
]

for col in cols_range:
    parts = split_range(df2[col])
    for p in ['median', 'min', 'max']:
        df2[f"{col}_{p}"] = parts[p]
    df2.drop(columns=col, inplace=True)

# ✅ 5) حساب مؤشرات إضافية
df2["Coverage_calc_%"] = (
    100 * df2["Reported number of children receiving ART"] /
    df2["Estimated number of children needing ART based on WHO methods_median"]
)

df2["Treatment_gap"] = (
    df2["Estimated number of children needing ART based on WHO methods_median"] -
    df2["Reported number of children receiving ART"]
)

# ✅ 6) حفظ النسخة المنظّفة
clean_dir = r"E:\Corelia\Final Project\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "art_pediatric_clean_numeric.csv")
df2.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned pediatric ART file saved to: {output_path}")


✔ Cleaned pediatric ART file saved to: E:\Corelia\Final Project\clean\art_pediatric_clean_numeric.csv


In [9]:
import pandas as pd
import numpy as np
import os
import re

# 1) تحميل ملف الشيت الأول
adult_path = r"E:\Corelia\Final Project\art_coverage_by_country_clean.csv"
df1 = pd.read_csv(adult_path, encoding='utf-8')

# 2) تنظيف العمود الأساسي
df1["Reported number of people receiving ART"] = (
    df1["Reported number of people receiving ART"]
        .astype(str)
        .replace({"—": None, "Nodata": None, "No data": None, "no data": None})
        .str.replace(" ", "", regex=False)
        .str.replace(",", "", regex=False)
        .replace("nan", None)
        .astype(float)
)

# 3) دالة لفك الأعمدة اللي فيها رينج
def split_range(series):
    cleaned = (series.astype(str)
                       .str.replace("–", "-", regex=False)
                       .str.replace(" ", "", regex=False)
                       .str.replace(",", "", regex=False))
    pattern = r'(?P<median>\d+\.?\d*)\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    return cleaned.str.extract(pattern).astype(float)

# 4) الأعمدة اللي فيها رينج
cols_range = [
    "Estimated number of people living with HIV",
    "Estimated ART coverage among people living with HIV (%)"
]

for col in cols_range:
    parts = split_range(df1[col])
    for p in ['median', 'min', 'max']:
        df1[f"{col}_{p}"] = parts[p]
    df1.drop(columns=col, inplace=True)

# 5) حساب مؤشرات إضافية
df1["Coverage_calc_%"] = (
    100 * df1["Reported number of people receiving ART"] /
    df1["Estimated number of people living with HIV_median"]
)

df1["Treatment_gap"] = (
    df1["Estimated number of people living with HIV_median"] -
    df1["Reported number of people receiving ART"]
)

# 6) حفظ الملف باسم مخصص بدون كلمة adult
clean_dir = r"E:\Corelia\Final Project\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "art_coverage_clean_numeric.csv")
df1.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned file saved to: {output_path}")



✔ Cleaned file saved to: E:\Corelia\Final Project\clean\art_coverage_clean_numeric.csv


In [10]:
df3

,Country,Year,Count,Count_median,Count_min,Count_max,WHO Region
0,Afghanistan,2018,0.1[0.1–0.1],0.1,0.1,0.1,Eastern Mediterranean
1,Albania,2018,na,NaN,NaN,NaN,Europe
2,Algeria,2018,0.1[0.1–0.1],0.1,0.1,0.1,Africa
3,Angola,2018,2.0[1.7–2.3],2.0,1.7,2.3,Africa
4,Argentina,2018,0.4[0.4–0.4],0.4,0.4,0.4,Americas
...,...,...,...,...,...,...,...
675,Venezuela (Bolivarian Republic of),2000,na,NaN,NaN,NaN,Americas
676,Viet Nam,2000,0.3[0.2–0.3],0.3,0.2,0.3,Western Pacific
677,Yemen,2000,0.1[0.1–0.1],0.1,0.1,0.1,Eastern Mediterranean
678,Zambia,2000,16.2[14.3–18.2],16.2,14.3,18.2,Africa


In [11]:
import pandas as pd
import numpy as np
import os
import re

# ✅ 1) تحميل الملف
cases_path = r"E:\Corelia\Final Project\no_of_cases_adults_15_to_49_by_country_clean.csv"
df_cases = pd.read_csv(cases_path, encoding='utf-8')

# ✅ 2) تنظيف عمود Count (لو فيه قيم مش رقمية أو "na")
df_cases["Count"] = (
    df_cases["Count"]
        .astype(str)
        .replace({"—": None, "Nodata": None, "na": None, "Na": None, "NaN": None})
        .str.replace("–", "-", regex=False)   # لو فيه dash غريبة
        .str.replace(" ", "", regex=False)    # حذف الفراغات
        .replace("nan", None)
)

# ✅ 3) استخراج الأرقام من Count إلى 3 أعمدة: median, min, max
def split_range(series):
    pattern = r'(?P<median>\d+\.?\d*)\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    return series.str.extract(pattern).astype(float)

count_parts = split_range(df_cases["Count"])
df_cases["Count_median"] = count_parts["median"]
df_cases["Count_min"] = count_parts["min"]
df_cases["Count_max"] = count_parts["max"]

# ✅ 4) حذف عمود Count الأصلي (اختياري)
# df_cases.drop(columns=["Count"], inplace=True)

# ✅ 5) حفظ الملف المنظف
clean_dir = r"E:\Corelia\Final Project\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "no_of_cases_adults_15_49_clean_numeric.csv")
df_cases.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned file saved to: {output_path}")


✔ Cleaned file saved to: E:\Corelia\Final Project\clean\no_of_cases_adults_15_49_clean_numeric.csv


In [14]:
import pandas as pd
import numpy as np
import os
import re

# ✅ 1) تنظيف عمود Count
df3["Count"] = (
    df3["Count"]
        .astype(str)
        .replace({
            "—": None,
            "Nodata": None,
            "na": None,
            "Na": None,
            "NaN": None
        })
        .str.replace("–", "-", regex=False)   # تصليح الشرطة
        .str.replace(" ", "", regex=False)    # إزالة المسافات
        .replace("nan", None)
)

# ✅ 2) استخراج median، min، max من عمود Count
def split_range(series):
    pattern = r'(?P<median>\d+\.?\d*)\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    return series.str.extract(pattern).astype(float)

count_parts = split_range(df3["Count"])
df3["Count_median"] = count_parts["median"]
df3["Count_min"] = count_parts["min"]
df3["Count_max"] = count_parts["max"]

# ✅ 3) حفظ النسخة المنظفة
clean_dir = r"E:\Corelia\Final Project\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "no_of_cases_adults_15_49_clean_numeric.csv")
df3.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned df3 saved to: {output_path}")


✔ Cleaned df3 saved to: E:\Corelia\Final Project\clean\no_of_cases_adults_15_49_clean_numeric.csv


In [15]:
df4

,Country,Year,Count,Count_median,Count_min,Count_max,WHO Region
0,Afghanistan,2018,500[200–610],500.0,200.0,610.0,Eastern Mediterranean
1,Albania,2018,na,NaN,NaN,NaN,Europe
2,Algeria,2018,200[200–200],200.0,200.0,200.0,Africa
3,Angola,2018,14000[9500–18000],14000.0,9500.0,18000.0,Africa
4,Argentina,2018,1700[1300–2100],1700.0,1300.0,2100.0,Americas
...,...,...,...,...,...,...,...
505,Venezuela (Bolivarian Republic of),2000,na,NaN,NaN,NaN,Americas
506,Viet Nam,2000,6100[4300–7800],6100.0,4300.0,7800.0,Western Pacific
507,Yemen,2000,100[100–200],100.0,100.0,200.0,Eastern Mediterranean
508,Zambia,2000,62000[49000–81000],62000.0,49000.0,81000.0,Africa


In [16]:
import pandas as pd
import numpy as np
import os
import re

# ✅ 1) تحميل الملف
df4 = pd.read_csv(r'E:\Corelia\Final Project\no_of_deaths_by_country_clean.csv', encoding='utf-8')

# ✅ 2) تنظيف عمود Count (لأن فيه "[...]" وممكن رموز غير مفهومة)
df4["Count"] = (
    df4["Count"]
        .astype(str)
        .replace({"—": None, "na": None, "NaN": None, "Nodata": None, "No data": None})
        .str.replace("–", "-", regex=False)   # توحيد الشرطة
        .str.replace(" ", "", regex=False)
        .replace("nan", None)
)

# ✅ 3) استخراج median / min / max من Count
def split_range(series):
    pattern = r'(?P<median>\d+\.?\d*)\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    return series.str.extract(pattern).astype(float)

count_parts = split_range(df4["Count"])

# ✅ 4) تعويض الأعمدة الموجودة أو المفقودة (لو فيها NaN)
df4["Count_median"] = df4["Count_median"].combine_first(count_parts["median"])
df4["Count_min"]    = df4["Count_min"].combine_first(count_parts["min"])
df4["Count_max"]    = df4["Count_max"].combine_first(count_parts["max"])

# ✅ 5) حفظ النسخة المنظفة
clean_dir = r"E:\Corelia\Final Project\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "no_of_deaths_clean_numeric.csv")
df4.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned df4 saved to: {output_path}")


✔ Cleaned df4 saved to: E:\Corelia\Final Project\clean\no_of_deaths_clean_numeric.csv


In [17]:
df5

,Country,Year,Count,Count_median,Count_min,Count_max,WHO Region
0,Afghanistan,2018,7200[4100–11000],7200.0,4100.0,11000.0,Eastern Mediterranean
1,Albania,2018,na,NaN,NaN,NaN,Europe
2,Algeria,2018,16000[15000–17000],16000.0,15000.0,17000.0,Africa
3,Angola,2018,330000[290000–390000],330000.0,290000.0,390000.0,Africa
4,Argentina,2018,140000[130000–150000],140000.0,130000.0,150000.0,Americas
...,...,...,...,...,...,...,...
675,Venezuela (Bolivarian Republic of),2000,na,NaN,NaN,NaN,Americas
676,Viet Nam,2000,120000[110000–130000],120000.0,110000.0,130000.0,Western Pacific
677,Yemen,2000,1100[680–2500],1100.0,680.0,2500.0,Eastern Mediterranean
678,Zambia,2000,890000[800000–1000000],890000.0,800000.0,1000000.0,Africa


In [18]:
import pandas as pd
import numpy as np
import os
import re

# ✅ 1) تحميل الملف
df5 = pd.read_csv(r'E:\Corelia\Final Project\no_of_people_living_with_hiv_by_country_clean.csv', encoding='utf-8')

# ✅ 2) تنظيف عمود Count
df5["Count"] = (
    df5["Count"]
        .astype(str)
        .replace({"—": None, "na": None, "NaN": None, "Nodata": None, "No data": None})
        .str.replace("–", "-", regex=False)
        .str.replace(" ", "", regex=False)
        .replace("nan", None)
)

# ✅ 3) استخراج median / min / max من Count
def split_range(series):
    pattern = r'(?P<median>\d+\.?\d*)\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    return series.str.extract(pattern).astype(float)

count_parts = split_range(df5["Count"])

# ✅ 4) تعويض الأعمدة لو فيها قيم ناقصة
df5["Count_median"] = df5["Count_median"].combine_first(count_parts["median"])
df5["Count_min"]    = df5["Count_min"].combine_first(count_parts["min"])
df5["Count_max"]    = df5["Count_max"].combine_first(count_parts["max"])

# ✅ 5) حفظ الملف المنظف
clean_dir = r"E:\Corelia\Final Project\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "no_of_people_living_with_hiv_clean_numeric.csv")
df5.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned df5 saved to: {output_path}")


✔ Cleaned df5 saved to: E:\Corelia\Final Project\clean\no_of_people_living_with_hiv_clean_numeric.csv


In [19]:
df6

,Country,Received Antiretrovirals,Needing antiretrovirals,Percentage Recieved,Needing antiretrovirals_median,Needing antiretrovirals_min,Needing antiretrovirals_max,Percentage Recieved_median,Percentage Recieved_min,Percentage Recieved_max,WHO Region
0,Afghanistan,20,200[100–500],11[7–18],200.0,100.0,500.0,11.0,7.0,18.0,Eastern Mediterranean
1,Albania,No data,Nodata,Nodata,NaN,NaN,NaN,NaN,NaN,NaN,Europe
2,Algeria,320,500[500–500],74[69–78],500.0,500.0,500.0,74.0,69.0,78.0,Africa
3,Angola,9600,25000[19000–32000],38[29–48],25000.0,19000.0,32000.0,38.0,29.0,48.0,Africa
4,Argentina,1800,1800[1600–2000],95[85–95],1800.0,1600.0,2000.0,95.0,85.0,95.0,Americas
...,...,...,...,...,...,...,...,...,...,...,...
165,Venezuela (Bolivarian Republic of),410,Nodata,Nodata,NaN,NaN,NaN,NaN,NaN,NaN,Americas
166,Viet Nam,1900,2400[2000–2800],81[69–95],2400.0,2000.0,2800.0,81.0,69.0,95.0,Western Pacific
167,Yemen,30,500[200–500],13[8–20],500.0,200.0,500.0,13.0,8.0,20.0,Eastern Mediterranean
168,Zambia,56 500,48000[38000–57000],95[94–95],48000.0,38000.0,57000.0,95.0,94.0,95.0,Africa


In [20]:
import pandas as pd
import numpy as np
import os
import re

# ✅ 1) تحميل الملف
df6 = pd.read_csv(r'E:\Corelia\Final Project\prevention_of_mother_to_child_transmission_by_country_clean.csv', encoding='utf-8')

# ✅ 2) تنظيف عمود "Received Antiretrovirals"
df6["Received Antiretrovirals"] = (
    df6["Received Antiretrovirals"]
        .astype(str)
        .replace({"—": None, "na": None, "NaN": None, "Nodata": None, "No data": None})
        .str.replace(",", "", regex=False)
        .str.replace(" ", "", regex=False)
        .replace("nan", None)
        .astype(float)
)

# ✅ 3) تنظيف الأعمدة النصية اللي فيها range
def split_range(series):
    pattern = r'(?P<median>\d+\.?\d*)\[(?P<min>\d+\.?\d*)-(?P<max>\d+\.?\d*)\]'
    return series.str.extract(pattern).astype(float)

# استخراج الأرقام من العمودين:
need_parts = split_range(df6["Needing antiretrovirals"].astype(str).str.replace(" ", "", regex=False))
percent_parts = split_range(df6["Percentage Recieved"].astype(str).str.replace(" ", "", regex=False))

# ✅ 4) تعويض القيم الناقصة إن وُجدت
df6["Needing antiretrovirals_median"] = df6["Needing antiretrovirals_median"].combine_first(need_parts["median"])
df6["Needing antiretrovirals_min"]    = df6["Needing antiretrovirals_min"].combine_first(need_parts["min"])
df6["Needing antiretrovirals_max"]    = df6["Needing antiretrovirals_max"].combine_first(need_parts["max"])

df6["Percentage Recieved_median"]     = df6["Percentage Recieved_median"].combine_first(percent_parts["median"])
df6["Percentage Recieved_min"]        = df6["Percentage Recieved_min"].combine_first(percent_parts["min"])
df6["Percentage Recieved_max"]        = df6["Percentage Recieved_max"].combine_first(percent_parts["max"])

# ✅ 5) حفظ النسخة المنظفة
clean_dir = r"E:\Corelia\Final Project\clean"
os.makedirs(clean_dir, exist_ok=True)

output_path = os.path.join(clean_dir, "mother_to_child_transmission_clean_numeric.csv")
df6.to_csv(output_path, index=False, encoding="utf-8")

print(f"✔ Cleaned df6 saved to: {output_path}")


✔ Cleaned df6 saved to: E:\Corelia\Final Project\clean\mother_to_child_transmission_clean_numeric.csv


In [21]:
df6

,Country,Received Antiretrovirals,Needing antiretrovirals,Percentage Recieved,Needing antiretrovirals_median,Needing antiretrovirals_min,Needing antiretrovirals_max,Percentage Recieved_median,Percentage Recieved_min,Percentage Recieved_max,WHO Region
0,Afghanistan,20.0,200[100–500],11[7–18],200.0,100.0,500.0,11.0,7.0,18.0,Eastern Mediterranean
1,Albania,NaN,Nodata,Nodata,NaN,NaN,NaN,NaN,NaN,NaN,Europe
2,Algeria,320.0,500[500–500],74[69–78],500.0,500.0,500.0,74.0,69.0,78.0,Africa
3,Angola,9600.0,25000[19000–32000],38[29–48],25000.0,19000.0,32000.0,38.0,29.0,48.0,Africa
4,Argentina,1800.0,1800[1600–2000],95[85–95],1800.0,1600.0,2000.0,95.0,85.0,95.0,Americas
...,...,...,...,...,...,...,...,...,...,...,...
165,Venezuela (Bolivarian Republic of),410.0,Nodata,Nodata,NaN,NaN,NaN,NaN,NaN,NaN,Americas
166,Viet Nam,1900.0,2400[2000–2800],81[69–95],2400.0,2000.0,2800.0,81.0,69.0,95.0,Western Pacific
167,Yemen,30.0,500[200–500],13[8–20],500.0,200.0,500.0,13.0,8.0,20.0,Eastern Mediterranean
168,Zambia,56500.0,48000[38000–57000],95[94–95],48000.0,38000.0,57000.0,95.0,94.0,95.0,Africa
